In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def encode(train, test):
    label_encoder = LabelEncoder().fit(train.classID)
    labels = label_encoder.transform(train.classID)
    classes = list(label_encoder.classes_)

    train = train.drop(['class', 'slice_file_name'], axis=1)
    test = test.drop('slice_file_name', axis=1)

    return train, labels, test, classes

train, labels, test, classes = encode(train, test)

# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]
    

nb_features = 64 # number of features per features type (shape, texture, margin)   
nb_class = len(classes)

# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

# Keras model with one Convolution1D layer
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

ValueError: could not broadcast input array from shape (6371,6) into shape (6371,64)

In [35]:
import csv
import librosa.display
import numpy as np
import matplotlib.pyplot as pyplot
import soundfile as sf

first_one = 1
counter = 0
with open( "UrbanSound8K.csv", "r") as dataset_info:
    lines = csv.reader(dataset_info)

    for row in lines:
        if first_one == 1:
            first_one = 0
            continue
        sound_name = ""
        sound_name = row[0].replace(".wav", "") #Changing the name of the sound file
        sound_file_temp, sample_rate = sf.read("fold 1\\" + sound_name + ".wav", dtype="float32")
        sound_file_temp = sound_file_temp.T
        sound_file_temp2 = librosa.resample(sound_file_temp, sample_rate, 44100) #Resampling
        sound_file = librosa.to_mono(sound_file_temp2) #Resampling

        print (counter)
        counter = counter + 1

RuntimeError: Error opening 'fold 1\\100032-3-0-0.wav': System error.

In [ ]:
import pandas as pd
import numpy as np
import time
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D

start = time.time()

#Convert train and test data to dataframes
train_dataframe = pd.read_csv("train.csv", dtype=str)
test_dataframe = pd.read_csv("test.csv", dtype=str)

#Renaming
for index, row in train_dataframe.iterrows():
    row["slice_file_name"] = row["slice_file_name"].replace(".wav", "")
    row["slice_file_name"] = row["slice_file_name"] + ".png"

for index, row in test_dataframe.iterrows():
    row["slice_file_name"] = row["slice_file_name"].replace(".wav", "")
    row["slice_file_name"] = row["slice_file_name"] + ".png"

#Keras image generator
image_data_generator = ImageDataGenerator(rescale=1./255., validation_split = 0.125)

train_generator = image_data_generator.flow_from_dataframe(
    dataframe = train_dataframe,
    directory = "images1\\",
    x_col = "slice_file_name",
    y_col = "class",
    subset = "training",
    batch_size = 64,
    seed = 51, #Random random
    shuffle = True, #Shuffling
    class_mode = "input",
    target_size = (32, 32))

validation_generator = image_data_generator.flow_from_dataframe(
    dataframe = train_dataframe,
    directory = "images1\\",
    x_col = "slice_file_name",
    y_col = "class",
    subset = "validation",
    batch_size = 64,
    seed = 51,
    shuffle = True,
    class_mode = "input",
    target_size = (32, 32))
print(train_generator[0][0].shape)
train_generator2 = np.zeros_like(train_generator)
print(train_generator2)
i = 0
j = 0
count = 0
for i in range(86):
       for j in range(2):
            train_generator2[i][j] = train_generator[i][j][:, :, 0, :]
           

Using TensorFlow backend.


Found 5489 images.
Found 784 images.
(64, 32, 32, 3)


In [38]:
input_shape = (32, 3)                         
model_input = Input(shape=input_shape, name='input')                    
layer = model_input                                                     
layer = Conv1D(filters=40,
               kernel_size=10,
               padding='same',
               activation='relu')(layer)
layer = BatchNormalization(axis=2)(layer)                           
layer = MaxPooling1D(pool_size=40,
                     padding='same')(layer)                             
layer = Dropout(0.2)(layer)  

layer = Dense(40, activation = 'relu')(layer)
layer = BatchNormalization(axis = 2)(layer)
#layer = keras.layers.Flatten(input_shape=input_shape) (layer)
model_output = Dense(2, activation='relu')(layer)
model = models.Model(inputs=model_input, outputs=model_output)
opt = optimizers.Adam(0.00001)

model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 20, 3)             0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 20, 40)            1240      
_________________________________________________________________
batch_normalization_29 (Batc (None, 20, 40)            160       
_________________________________________________________________
max_pooling1d_15 (MaxPooling (None, 1, 40)             0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 1, 40)             0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 1, 40)             12960     
_________________________________________________________________
dropout_30 (Dropout)         (None, 1, 40)             0         
__________

In [55]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size

model.fit_generator(generator=train_generator, #Fitting the generator to model
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10) #Gradient
model.evaluate_generator(generator=validation_generator, steps=STEP_SIZE_VALID)

end = time.time()
print("Total time: ", end - start, " seconds")

Epoch 1/10


ValueError: Error when checking input: expected input to have 3 dimensions, but got array with shape (64, 256, 256, 3)